# Custom Pipeline using MIPI as input source
---

<div class="alert alert-box alert-info">
Please use Jupyter labs http://&lt;board_ip_address&gt;/lab for this notebook.
</div>

This notebook shows your how to create a custom pipeline that uses the [Digilent Pcam 5C Camera](https://reference.digilentinc.com/reference/add-ons/pcam-5c/start) as input video source

## Aims
* Use Pcam 5C as input video source
* Compose branched pipeline on the input video
* Enhance the output by using a simple threshold

## Table of Contents
* [Download Composable Overlay](#download)
* [Start Pcam 5C and HDMI out video pipeline](#start_video)
* [Branched Pipeline](#branched)
* [Enhance the Output](#enhance)
* [Stop Video Pipeline](#stop_hdmi)
* [Conclusion](#conclusion)

----

## Revision History

* v1.0 | 15 June 2021 | First notebook revision.
* v1.1 | 11 August 2021 | Update notebook to composable overlay API 1.0.0

----

## Download Composable Overlay <a class="anchor" id="download"></a>

Import the pynq video libraries as well as Composable class and the drivers for the IP.

Download the Composable Overlay using `pynq.Overlay` and grab a handler to the `composable` hierarchy

In [ ]:
from pynq import Overlay
from pynq.lib.video import *
from pynq_composable import *
import numpy as np

ol = ComposableOverlay("../overlay/cv_dfx_4_pr.bit")

cpipe = ol.composable

## Start Pcam 5C and HDMI out video <a class="anchor" id="start_video"></a>

Get `HDMIVideo` object and start video

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to connect HDMI out cables to a valid screen and to connect Pcam 5C may cause the notebook to hang
</div>

In [ ]:
video = HDMIVideo(ol, 'MIPI')
video.start()

## Branched Pipeline <a class="anchor" id="branched"></a>

In this part of the notebook, we will bring new functionality into the four DFX regions to compose the [Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians) application that was also introduced in the previous session. 

Load dynamic IP

In [ ]:
cpipe.loadIP(['pr_fork/duplicate_accel', 'pr_join/subtract_accel', 'pr_0/filter2d_accel'])

Grab handlers and set up default values

In [ ]:
filter2d = cpipe.filter2d_accel
duplicate = cpipe.pr_fork.duplicate_accel
subtract = cpipe.pr_join.subtract_accel
filter2d_d = cpipe.pr_0.filter2d_accel

filter2d.sigma = 0.3
filter2d.kernel_type = 'gaussian_blur'

filter2d_d.sigma = 12
filter2d_d.kernel_type = 'gaussian_blur'

Grab handlers to the MIPI camera

In [ ]:
mipi_in = cpipe.mipi_in
mipi_out = cpipe.mipi_out

The Difference of Gaussians is realized by subtracting one Gaussian blurred version of an original image from another less blurred version of the original. In the Composable Overlay this is achieved by branching the pipeline, which is expressed as a list of a list.

In [ ]:
video_pipeline = [mipi_in, filter2d, duplicate, [[filter2d_d], [1]], subtract, mipi_out]

cpipe.compose(video_pipeline)

cpipe.graph

## Enhance the Output <a class="anchor" id="enhance"></a>

Depending on light condition and ambient the result of the Difference of Gaussians can look weak. We are going to use the LUT IP to set a threshold to improve how the output results.

Grab a handler to the LUT IP and configure the numpy array with the thresholding values

In [ ]:
lut = cpipe.lut_accel
rgb2gray = cpipe.rgb2gray_accel

Set up a simple threshold. Any channel in the input pixels with intensity above 11 will be mapped to an intensity of 255 in the output.

In [ ]:
thr = np.array([np.ones((3,3), dtype=np.uint8) * 11 , np.ones((3,3), dtype=np.uint8) * 255],dtype=np.uint8)
lut.threshold = thr

In [ ]:
cpipe.insert(([rgb2gray,lut], 5))

cpipe.graph

You can also watch the output in color by removing the `rgb2gray` IP

In [ ]:
cpipe.remove([rgb2gray])

## Stop Video Pipeline <a class="anchor" id="stop_hdmi"></a>

Finally stop the HDMI video pipeline

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

Failure to stop the HDMI Video may hang the board 
when trying to download another bitstream onto the FPGA
</div>

In [ ]:
video.stop()

----

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has shown how to create a custom pipeline using the Pcam 5C as input video source and how the output can be enhanced by a simple thresholding

[⬅️ Advanced Features](07_advanced_features.ipynb) | |

Copyright &copy; 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----